# Custom Chatbot Project

### Fashion Trends Datase

The dataset used for the project is the fashion trends dataset. 

The dataset seems bit challanging but not too complex, it just needed some data wrangling to clean the text column and git ride of the un needed ones, I removed the repeated words like "2023 Fashion Trend: " and the special characters to have a clean dataset without noises

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [32]:
import pandas as pd
import numpy as np
import ast
import tiktoken
from sklearn.metrics.pairwise import cosine_distances
pd.set_option('display.max_colwidth', None)
df = pd.read_csv("data/2023_fashion_trends.csv")[['Trends']]
df.columns= ['text']

In [33]:
df.head(30)

,text
0,"2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that would look just as good in head-to-toe looks (see: a pantsuit) as accent accessory pieces (shoes, handbags, jewelry)."
1,"2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For spring, this trend is translated into tailored silhouettes, interesting pocket placements, elevated fabrics like silk and organza, and colors that go beyond khaki and olive."
2,"2023 Fashion Trend: Sheer Clothing. ""Bare it all"" has been the motto since the end of the lockdown. In 2023, naked dressing makes its way from the red carpet – where celebrities like Cher and Rihanna have been sporting the trend forever – to street style. From a cellophane-like dress, worn over a boldly hued maxi skirt at Tory Burch's spring 2023 show, to a frothy frock revealing undergarments at Victoria Beckham, the previously risqué trend is coming not only for your weekend wardrobe but even workwear."
3,"2023 Fashion Trend: Denim Reimagined. From double-waisted jeans to carpenter jeans, it's been a while since we were this excited about denim trends. It seems like even the most luxe runway designers agree, sending out strapless dresses, shirting, and even undergarments and shoes (thigh-high-boot-jean hybrids anyone?) in the material. Whatever category you decide on, opt for timeless cuts and silhouettes that can stay in your closet rotation once the novelty wears off."
4,"2023 Fashion Trend: Shine For The Daytime. The amount of shine on the 2023 runways would make you think we are back in the Roaring '20s. While sequins may feel over as soon as the clock strikes midnight on January 31, this year, designers are making a case for ""daytime shine"" – wearable separates that can spice up even the most basic staples like tank tops (see above) and blue jeans. While this trend comes in all metallic shades (gold, bronze, rose gold etc.), we are partial to the liquid silver look that can double as a neutral."
5,"2023 Fashion Trend: Maxi Skirts. In response to the ultra unpractical mini skirts of 2022, maxi skirts are here to dominate the year. In line with the aforementioned cargo and denim trends, expect lots of low-rise, pocket-adorned silhouettes and floor-grazing jean skirts, as well as styles in various prints and unexpected materials like velvet. Long live long hemlines!"
6,"2023 Fashion Trend: Cobalt Blue. The strongest color story to come out of Spring 2023 runways, cobalt blue has burst through the collections with the freshness of a sea mist on a morning day. Just bright enough to warrant a double take, yet subtle enough to be worked into daily wear, it's the type of deep blue that will excite even the most color-averse. Bonus points: It pairs well with Pantone's Viva Magenta."
7,"Sculptural Bags That Make a Simple Statement. Under the accessories category, CEO and founder of Caitlin Jaymes Organizing & Styling, Caitlyn Jaymes is honing in on sculptural bags that are simple enough to be considered versatile but still make a statement.\n\n“[For everyday use], I would purchase it in camel, ivory, or, if I wanted a color option, I would purchase pale green or pale yellow,” she tells InStyle. The style expert goes on to say she could style the staple item in a myriad of ways, but for a casual look, something along the lines of a “straight leg jeans, light knit cardigan, and a white slim sneaker” combo would be her go-to. For dressier occasions, she tells the dot com she’d likely “pair a slip satin long skirt with a white ribbed tank and pointed toe kitten heels.”"
8,"All of the Pastels. Pastels are classic (albeit slightly cliche) for spring, but they’re often given a bad rap for feeling a bit cheesy. Personal stylist Allie McKenna is here to tell

In [34]:
df['text'] = df['text'].str.replace('"', '')
df['text'] = df['text'].str.replace('-', '')
df['text'] = df['text'].str.replace('—', ' ')
df['text'] = df['text'].str.replace('2023 Fashion Trend: ', '')

In [35]:
x =df.iloc[20].to_string()
print(x)

text    Moto Detailing. You’ll probably never become a motocross racer, but if the spring runways are any indication, it’s worth dressing like one. Racing jackets, mixed leather, and buckle details abound this season, adding a highoctane edge to any ’fit no actual bikes necessary.


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [36]:
#from openai import OpenAI
from openai import OpenAI
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key = "voc-1060074748126677387809167c0cc4388e176.51105398"
)


In [53]:
def compare_answrs(question,model):
    query = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content":question}]

    response = client.chat.completions.create(
        model= model ,#"gpt-3.5-turbo",  # Use gpt-4-turbo or gpt-3.5-turbo
        messages = query ,
        temperature=0 
    )
    print("Question: "+question)
    response = response.dict()
    print("##########################################################################")
    print("Answer using the default model: "+response["choices"][0]["message"]["content"])
    embedding_model="text-embedding-ada-002"
    response= client.embeddings.create(model=embedding_model,input=df["text"].tolist())
    response= response.dict()
    embeddings = [data["embedding"] for data in response["data"]]
    df["embeddings"]=embeddings
    question_embedding = client.embeddings.create(model=embedding_model,input=question)
    question_embedding =dict(question_embedding)
    question_embedding=question_embedding["data"][0]
    question_embedding = dict(question_embedding)["embedding"]
    question_embedding = np.array(question_embedding).reshape(1, -1)
    distances = cosine_distances(question_embedding, df["embeddings"].tolist())[0]
    df["distances"]=distances
    df.sort_values(by="distances",inplace=True)
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokenizer.encode(question)
    prompt_template = """
    Answer the question based on the context below, and if the question
    can't be answered based on the context, say "I don't know"
    
    Context: 
    
    {}
    
    ---
    
    Question: {}
    Answer:"""
    question_tokens_cnt = len(tokenizer.encode(question))
    context_tokens_cnt = len(tokenizer.encode(prompt_template))
    current_token_cnt = question_tokens_cnt + context_tokens_cnt
    context = []
    for i in df["text"].values:
        current_token_cnt += len(tokenizer.encode(i))
        if current_token_cnt <= 1000:
          context.append(i)
        else:
            break   
    prompt_template = prompt_template.format("\n\n###\n\n".join(context),question)
    
    query2 = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content":prompt_template}]

    response2 = client.chat.completions.create(
    model=model,  # Use gpt-4-turbo or gpt-3.5-turbo
    messages = query2 ,
    temperature=0 
    )
    response2 = response2.dict()
    response2 =response2["choices"][0]["message"]["content"]
    print("##########################################################################")
    print("Answer using the custom context: "+response2)


    

 Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that would look just as good in head-to-toe looks (see: a pantsuit) as accent accessory pieces (shoes, handbags, jewelry).

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [56]:
question ="Which fashion trend dominated the conversations for Spring/Summer 2023, and how did Molly Goddard interpret it?"
model = "gpt-3.5-turbo"
compare_answrs(question,model)

Question: Which fashion trend dominated the conversations for Spring/Summer 2023, and how did Molly Goddard interpret it?
##########################################################################
Answer using the default model: The fashion trend that dominated conversations for Spring/Summer 2023 was sustainability and eco-consciousness. Designers and brands focused on creating more environmentally friendly collections and promoting ethical practices within the industry.

Molly Goddard interpreted this trend by incorporating sustainable materials and practices into her designs. She used organic cotton, recycled fabrics, and eco-friendly dyes to create her collection. Additionally, she emphasized the importance of transparency in her supply chain and production process to ensure that her garments were made ethically and with minimal impact on the environment. Overall, Molly Goddard's interpretation of the sustainability trend for Spring/Summer 2023 showcased her commitment to creating 

C:\Users\ahmed\AppData\Local\Temp\ipykernel_41292\3362761342.py:12: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response = response.dict()
C:\Users\ahmed\AppData\Local\Temp\ipykernel_41292\3362761342.py:17: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response= response.dict()


##########################################################################
Answer using the custom context: Sheer fashion dominated the conversations for Spring/Summer 2023. Molly Goddard interpreted this trend with her romantic take, featuring her signature ruffle details on sheer orange midi dresses.


C:\Users\ahmed\AppData\Local\Temp\ipykernel_41292\3362761342.py:63: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response2 = response2.dict()


### Question 2

In [55]:
question ="What are the trending fashion items for this spring, according to stylist Naina Singla?"
model = "gpt-3.5-turbo"
compare_answrs(question,model)

Question: What are the trending fashion items for this spring, according to stylist Naina Singla?
##########################################################################
Answer using the default model: I'm sorry, but I don't have real-time access to the latest trends from stylist Naina Singla. However, some general trends for spring 2021 include pastel colors, oversized blazers, statement sleeves, crochet pieces, and wide-leg pants. You can check Naina Singla's website or social media for more specific and up-to-date fashion recommendations.


C:\Users\ahmed\AppData\Local\Temp\ipykernel_41292\3362761342.py:12: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response = response.dict()
C:\Users\ahmed\AppData\Local\Temp\ipykernel_41292\3362761342.py:17: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response= response.dict()


##########################################################################
Answer using the custom context: Statement maxi skirts are the trending fashion items for this spring, according to stylist Naina Singla.


C:\Users\ahmed\AppData\Local\Temp\ipykernel_41292\3362761342.py:63: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response2 = response2.dict()
